In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange


In [2]:
data = pd.read_csv("Data.csv")
data

,Unnamed: 0,Doc. Date,Material,Order qty,Ship-to nu,PCS delivered,HL delivered,delivery_days,delivery_flag,lead_time_creation_vs_rdd_flag,...,Container Size,Variétés,Segment LE,Latitude,Longitude,Degre Alc,TTC,Brut + TE,Net + TE - Hors majoration de rompu de palette,DA
0,0,2021-01-11 00:00:00,10946,5.0,29606863,350.0,21.0,7,1,1,...,"6,000 L",Blonde,PREMIUM,50.6455,2.9619,6.6,18.137448,15.582,15.114540,0.0
1,1,2021-01-11 00:00:00,19898,2.0,29606863,140.0,8.4,7,1,1,...,NaN,NaN,NaN,50.6455,2.9619,5.0,17.396467,14.688,14.497056,0.0
2,2,2021-01-11 00:00:00,80176,4.0,29606863,280.0,16.8,7,1,1,...,NaN,NaN,NaN,50.6455,2.9619,0.0,12.279859,10.368,10.233216,0.0
3,3,2021-01-11 00:00:00,66989,1.0,29606863,70.0,4.2,7,1,1,...,"6,000 L",-,SUPER PREMIUM,50.6455,2.9619,8.0,21.996000,18.330,18.330000,0.0
4,4,2021-01-08 00:00:00,11766,70.0,29378784,70.0,4.2,6,1,1,...,NaN,NaN,NaN,50.6978,2.2423,5.2,17.359200,14.466,14.466000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73665,73665,2019-12-30 00:00:00,17028,2.0,29446255,140.0,8.4,8,1,1,...,"6,000 L",Rituel,PREMIUM,47.7993,-0.7210,9.0,23.575622,20.856,19.646352,0.0
73666,73666,2019-12-30 00:00:00,33335,2.0,29446255,140.0,8.4,8,1,1,...,"6,000 L",Rosée,PREMIUM,47.7993,-0.7210,3.0,19.770696,17.490,16.475580,0.0
73667,73667,2019-12-30 00:00:00,53566,2.0,29446255,140.0,8.4,8,1,1,...,"6,000 L",IPA,PREMIUM,47.7993,-0.7210,7.5,23.575622,20.856,19.646352,0.0
73668,73668,2019-12-30 00:00:00,63482,1.0,29446255,70.0,4.2,8,1,1,...,NaN,NaN,NaN,47.7993,-0.7210,5.1,14.576040,12.786,12.146700,0.0


In [3]:
u=[]; l1=[]; l2=[]
for i in data["Ship-to nu"].unique():
    u.append(i)
    l1.append((data[data["Ship-to nu"]==i]["Latitude"]).iloc[0])
    l2.append((data[data["Ship-to nu"]==i]["Longitude"]).iloc[0])


dloc = pd.DataFrame()
dloc["Wholesaler"] = u
dloc["Latitude"] = l1
dloc["Longitude"] = l2

In [4]:
# dloc.to_csv("wholesaler_coordinates.csv", index=False)

In [5]:
from geopy.distance import great_circle
u = data["Ship-to nu"].unique()

foo={}
for i in tqdm(u):
    l=[]
    x = data[data["Ship-to nu"]==i]
    
    for j in u:
        
        y = data[data["Ship-to nu"]==j]
        
    #     print(x["Latitude"].iloc[0], x["Longitude"].iloc[0])
        
        l1 = (x["Latitude"].iloc[0], x["Longitude"].iloc[0])
        l2 = (y["Latitude"].iloc[0], y["Longitude"].iloc[0])
        
        
        try:
            dy = (great_circle(l1, l2).miles)
#             print(dy)
            l.append((dy, j))
        except:
            pass
            
    foo[str(i)] = [l]
    
    

100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [00:51<00:00,  3.46it/s]


In [6]:
dfx = pd.DataFrame(foo).T.reset_index()
dfx.columns=["Wholesaler", "Distance_in_miles"]

# sorted(dfx["Distance_in_miles"][0])
def sortfun(a):
    return sorted(a)

dfx["Distance_in_miles"] = dfx["Distance_in_miles"].apply(sortfun)
dfx

# dfx.to_csv("sorted_user_to_user_distance.csv", index=False)

,Wholesaler,Distance_in_miles
0,29606863,"[(0.0, 29606863), (5.532776754238337, 29379646..."
1,29378784,"[(0.0, 29378784), (3.7318753510029694, 2959551..."
2,29379853,"[(0.0, 29379853), (18.237506633013428, 2937982..."
3,29379855,"[(0.0, 29379855), (32.02405053051001, 29379858..."
4,29379861,"[(0.0, 29379861), (19.30666227837075, 29379820..."
...,...,...
174,29643421,"[(0.0, 29643421), (14.923809034154816, 2978013..."
175,29986364,"[(0.0, 29986364), (11.705556934074652, 2972295..."
176,40001857,"[(0.0, 40001857), (3.4544584010584827, 2937982..."
177,29983887,"[(0.0, 29983887), (14.663497221419183, 2937969..."


In [21]:
# dfx.to_csv("sorted_user_to_user_distance.csv", index=False)

In [20]:
data[data["Latitude"].isna()==True]["Ship-to nu"].unique()

array([29379632, 29563814, 29379841, 29379838, 29379667, 29563837,
       29378966, 29610633, 29801344], dtype=int64)